In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

dataset_path='/content/gdrive/MyDrive/TB-Dataset'
# infected_path='/content/gdrive/MyDrive/TB-Dataset/Infected'
IMAGE_SIZE =256

training_data=[]
# testing_data=[]

def manage_data(directory,Type=None):
  # Type=Type.lower()
  count=0
  CATEGORIES=['Normal','Infected']
  for category in CATEGORIES:
    label =CATEGORIES.index(category)
    path = os.path.join(directory,category)
    images=os.listdir(path)
    
    for img in images:
      image =cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
      image=cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))
      training_data.append([image,label])
      count +=1
      print(count,os.path.join(path,img),label)


      # else:
      #   testing_data.append([image,label])
      #   count +=1
      #   print(count)
    


manage_data(dataset_path)
print("Training len",len(training_data))
# manage_data(testing_path,'test')
# print("Testing len",len(testing_data))





In [ ]:
import pickle
pick = open('/content/gdrive/MyDrive/TB-Dataset/train_data.pickle','wb')
pickle.dump(training_data,pick) 
pick.close()


In [ ]:
saved_data = open("/content/gdrive/MyDrive/TB-Dataset/train_data.pickle","rb")
saved_data = pickle.load(saved_data)

In [ ]:
import random

random.shuffle(saved_data)

In [ ]:
import numpy as np
X_train=[]
Y_train=[]

for feature,label in saved_data:
  X_train.append(feature)
  Y_train.append(label)
X_train=np.array(X_train).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,1)
X_train =X_train/255.0
Y_train=np.array(Y_train)


In [ ]:
Y_train[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 1])

In [ ]:
from tensorflow import keras

checkpoint = keras.callbacks.ModelCheckpoint('Tuberclosis.h5',verbose=1,save_best_only=True)
callbacks =[
            keras.callbacks.EarlyStopping(patience=2,monitor='val_loss'),
            checkpoint
]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

# model = Sequential()

# model.add(Conv2D(64, (3, 3), input_shape=X_train.shape[1:]))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # model.add(Conv2D(256, (3, 3)))
# # model.add(Activation('relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(128))
# model.add(Activation('relu'))

# model.add(Dropout(.2))

# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(.2))


# model.add(Dense(1))
# model.add(Activation('sigmoid'))


model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=X_train.shape[1:]),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(1024, activation='relu'),
tf.keras.layers.Dropout(.2) ,
tf.keras.layers.Dense(512,activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=32, epochs=5, validation_split=0.2 , callbacks=callbacks)

Epoch 1/5
176/176 [==============================] - 24s 98ms/step - loss: 0.4805 - accuracy: 0.7985 - val_loss: 0.1969 - val_accuracy: 0.9230

Epoch 00001: val_loss improved from inf to 0.19692, saving model to Tuberclosis.h5
Epoch 2/5
176/176 [==============================] - 16s 94ms/step - loss: 0.1455 - accuracy: 0.9436 - val_loss: 0.0997 - val_accuracy: 0.9722

Epoch 00002: val_loss improved from 0.19692 to 0.09965, saving model to Tuberclosis.h5
Epoch 3/5
176/176 [==============================] - 16s 93ms/step - loss: 0.0626 - accuracy: 0.9770 - val_loss: 0.1022 - val_accuracy: 0.9658

Epoch 00003: val_loss did not improve from 0.09965
Epoch 4/5
176/176 [==============================] - 16s 93ms/step - loss: 0.0400 - accuracy: 0.9876 - val_loss: 0.0574 - val_accuracy: 0.9793

Epoch 00004: val_loss improved from 0.09965 to 0.05738, saving model to Tuberclosis.h5
Epoch 5/5
176/176 [==============================] - 16s 93ms/step - loss: 0.0343 - accuracy: 0.9880 - val_loss: 0.0

In [ ]:
import cv2
CATEGORIES = ["Normal", "Infected"]  # will use this to convert prediction num to string value

model = keras.models.load_model("/content/Tuberclosis.h5")


def prepare(filepath):
    # IMG_SIZE = 70  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)  # return the image with shaping that TF wants.

NameError: ignored

In [ ]:
test_path='/content/gdrive/MyDrive/TB-Dataset/testing-data'
for i in os.listdir(test_path):
  prediction = model.predict([prepare(os.path.join(test_path,i))])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT
  print(i,prediction[0][0],CATEGORIES[int(prediction[0][0])])


normal-1.jpg 1.0 Infected
normal-4.jpeg 1.0 Infected
normal-3.jpg 1.0 Infected
normal-2.jpg 1.0 Infected
normal-6.jpeg 1.0 Infected
normal-5.jpeg 1.0 Infected


In [ ]:
prediction

array([[1.]], dtype=float32)

In [ ]:
print(CATEGORIES[int(prediction[0][0])])

Infected


In [ ]:
cp  /content/Tuberclosis.h5 /content/gdrive/MyDrive/TB-Dataset/